In [3]:
import pandas as pd
import re
import os

def calculate_hourly_average_with_subject(file_path, timestamp_col):
    """
    Reads a CSV file, calculates hourly averages for numeric columns,
    includes the first value of non-numeric columns, removes 'Unnamed' columns,
    and adds a 'subject' column derived from the digits in the file name.
    Args:
        file_path (str): The path to the CSV file.
        timestamp_col (str): The name of the timestamp column.
    Returns:
        pandas.DataFrame: Hourly aggregated DataFrame with a 'subject' column.
    """
    try:
        df = pd.read_csv(file_path)
        df.columns = df.columns.str.strip()
        
        # Drop 'Unnamed' columns (like index columns)
        df = df.loc[:, ~df.columns.str.startswith('Unnamed')]
        
        # Convert timestamp column to datetime
        df[timestamp_col] = pd.to_datetime(df[timestamp_col])

        # Set index to timestamp
        df = df.set_index(timestamp_col)

        # Separate numeric and non-numeric columns
        numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
        non_numeric_cols = df.select_dtypes(exclude=['number']).columns.tolist()

        # Aggregation rules
        agg_dict = {col: 'mean' for col in numeric_cols}
        agg_dict.update({col: 'first' for col in non_numeric_cols})

        # Resample to hourly and aggregate
        hourly_df = df.resample('h').agg(agg_dict).reset_index()
        hourly_df = hourly_df.rename(columns={timestamp_col: 'hour_start'})

        # Round numeric values to 2 decimal places
        hourly_df[numeric_cols] = hourly_df[numeric_cols].round(2)

        # Extract subject number from file name
        subject_number = int(re.search(r'\d+', os.path.basename(file_path)).group())
        hourly_df.insert(0, 'subject', subject_number)

        return hourly_df

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# === Script Execution === #
if __name__ == "__main__":
    input_file_folder = "CGMacrosInputFiles"
    timestamp_column = "Timestamp"

    input_files = [
        "CGMacros-001", "CGMacros-002", "CGMacros-003", "CGMacros-004", "CGMacros-005",
        "CGMacros-006", "CGMacros-007", "CGMacros-008", "CGMacros-009", "CGMacros-010",
        "CGMacros-011", "CGMacros-012", "CGMacros-013", "CGMacros-014", "CGMacros-015",
        "CGMacros-016", "CGMacros-017", "CGMacros-018", "CGMacros-019", "CGMacros-020",
        "CGMacros-021", "CGMacros-022", "CGMacros-023", "CGMacros-026", "CGMacros-027",
        "CGMacros-028", "CGMacros-029", "CGMacros-030", "CGMacros-031", "CGMacros-032",
        "CGMacros-033", "CGMacros-034", "CGMacros-035", "CGMacros-036", "CGMacros-038",
        "CGMacros-039", "CGMacros-041", "CGMacros-042", "CGMacros-043", "CGMacros-044",
        "CGMacros-045", "CGMacros-046", "CGMacros-047", "CGMacros-048", "CGMacros-049"
    ]

    for input_file in input_files:
        full_path = os.path.join(input_file_folder, input_file + ".csv")
        result_df = calculate_hourly_average_with_subject(full_path, timestamp_column)

        if result_df is not None:
            output_file = os.path.join(input_file_folder, input_file + "_Hourly_Averaged_With_Subject.csv")
            result_df.to_csv(output_file, index=False)
            print(f"✅ Hourly averages saved to: {output_file}")


✅ Hourly averages saved to: CGMacrosInputFiles\CGMacros-001_Hourly_Averaged_With_Subject.csv
✅ Hourly averages saved to: CGMacrosInputFiles\CGMacros-002_Hourly_Averaged_With_Subject.csv
✅ Hourly averages saved to: CGMacrosInputFiles\CGMacros-003_Hourly_Averaged_With_Subject.csv
✅ Hourly averages saved to: CGMacrosInputFiles\CGMacros-004_Hourly_Averaged_With_Subject.csv
✅ Hourly averages saved to: CGMacrosInputFiles\CGMacros-005_Hourly_Averaged_With_Subject.csv
✅ Hourly averages saved to: CGMacrosInputFiles\CGMacros-006_Hourly_Averaged_With_Subject.csv
✅ Hourly averages saved to: CGMacrosInputFiles\CGMacros-007_Hourly_Averaged_With_Subject.csv
✅ Hourly averages saved to: CGMacrosInputFiles\CGMacros-008_Hourly_Averaged_With_Subject.csv
✅ Hourly averages saved to: CGMacrosInputFiles\CGMacros-009_Hourly_Averaged_With_Subject.csv
✅ Hourly averages saved to: CGMacrosInputFiles\CGMacros-010_Hourly_Averaged_With_Subject.csv
✅ Hourly averages saved to: CGMacrosInputFiles\CGMacros-011_Hourly_Ave

In [5]:
#consolidation of all files into one single file i.e. appending rows of all files into one sinlge file
import pandas as pd
import os
import glob

# Step 1: Set the directory containing all CSV files
folder_path = "CGMacrosOutputFiles"  # 🔁 Replace this with your actual folder path
output_file = "SubjectsHourlyAveragedCGMacros.csv"

# Step 2: Get a list of all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Step 3: Read and concatenate all files
all_dataframes = []

for file in csv_files:
    df = pd.read_csv(file)
    
    # Optionally: Add a filename/subject column (e.g., from "CGMacros-019.csv")
    # subject = int(re.search(r'\d+', os.path.basename(file)).group())
    # df['subject'] = subject
    
    all_dataframes.append(df)

# Step 4: Merge all into one DataFrame
merged_df = pd.concat(all_dataframes, ignore_index=True)

#merged_df = merged_df.drop(columns=['intensity'], errors='ignore')

# Step 5: Save to output file
merged_df.to_csv(output_file, index=False)

print(f"✅ Merged {len(csv_files)} files into '{output_file}'")

✅ Merged 45 files into 'SubjectsHourlyAveragedCGMacros.csv'
